In [8]:
#-----------------------------------------------------------------------------------
#   1_pickle_to_mat
# 
#   Construct the Haldane model (with flux threading) on a {8,3} PBC cluster built 
#   from the coset table of a given {8,8} PBC cluster. 
#
#   Code written by Canon Sun. Last updated 2024-09-04.
#-----------------------------------------------------------------------------------

import numpy as np
from math import pi
from numpy import linalg as LA
import sympy as sp
import pickle
from sympy.utilities.lambdify import lambdify
import time
import os
import scipy.io as sio
from scipy.io import savemat

from tqdm import tqdm

infile = './Hh_flux_G24.pickle'
order = 24 #Size of group. 24 for Hh_flux_G24.pickle and 48 for Hh_flux_G48.pickle


HAll=main()
#Save to MATLAB
mdic={'HAll':HAll}
savemat("HAll_12_L=10.mat",mdic)    
    
    





first loop: 100%|██████████| 10/10 [08:56<00:00, 53.70s/it]


# Functions

In [5]:
def Ham(phi1,phi2,phi3,phi4):
    # Retrieve Hamiltonian
    symbolphi = sp.symbols(f"phi0:{4}", real=True)  # symbols for flux phases
    with open(infile, 'rb') as inf: 
        symbolH = pickle.loads(inf.read())
        
    total = len(symbolH[:,0])
    H = lambdify([symbolphi], symbolH)
    
    return H([phi1,phi2,phi3,phi4])


In [7]:
def main():
    L = 10 #Dimension of flux grid
    size = order*16 #Size of Hamiltonian. There are 16 sublattices.
    HAll = np.zeros((L,L,size,size),dtype = 'complex_')
    
   
    for i in tqdm(range(L), desc='first loop'):
        for j in range(L):
            phi1=2*pi*i/L
            phi2=2*pi*j/L
            
            H=Ham(phi1,phi2,0,0) #Get all the Hamiltonians in the phi1-phi2 plane
            HAll[i,j,:,:]=H
            
    
    return HAll